In [1]:
import time
overall_start = time.time()         # <== Mark starting time
import os
from sys import argv, path
import sys
import datetime
import json
import importlib

In [5]:
import numpy as np
import pandas as pd
import os
import pickle
import torch
import torch.nn.functional as F

from lips import get_root_path
from lips.dataset.scaler.standard_scaler import StandardScaler
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
from lips.dataset.airfransDataSet import download_data

In [11]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = '../src/Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

In [15]:
root_dir = r"C:/Users/pc/Desktop/github/ml4physim/src/"
default_input_dir = root_dir + "Dataset"
default_output_dir = root_dir + "submission_attention/sample_result_submission"
default_program_dir = root_dir + "submission_attention"
default_submission_dir = root_dir + "submission_attention"
input_dir = default_input_dir
output_dir = default_output_dir
program_dir= default_program_dir
submission_dir= default_submission_dir
DIRECTORY_NAME = root_dir + 'Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = root_dir + "lips_logs.log"
BENCH_CONFIG_PATH = os.path.join(root_dir, "LIPS","configurations","airfoil","benchmarks","confAirfoil.ini") #Configuration file related to the benchmark
SIM_CONFIG_PATH = os.path.join(submission_dir,"config.ini")
SAVE_PATH = os.path.join(output_dir,"AirfRANSModel")

In [19]:
with open("C:/Users/pc/Desktop/github/ml4physim/", 'rb') as f:
    benchmark = pickle.load(f)

PermissionError: [Errno 13] Permission denied: 'C:/Users/pc/Desktop/github/ml4physim/'

In [20]:
# Load the required benchmark datasets, if it is not pickled, pickle it for future use
try:
    with open("C:/Users/pc/Desktop/github/ml4physim/benchmark.pkl", 'rb') as f:
        benchmark = pickle.load(f)
except:
    benchmark = AirfRANSBenchmark(benchmark_path=DIRECTORY_NAME,
                                  config_path=BENCH_CONFIG_PATH,
                                  benchmark_name=BENCHMARK_NAME,
                                  log_path=LOG_PATH)
    benchmark.load(path=DIRECTORY_NAME)
    with open('C:/Users/pc/Desktop/github/ml4physim/benchmark.pkl', 'wb') as f:
        pickle.dump(benchmark, f)

In [32]:
import datetime as dt
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from lips.dataset.scaler.standard_scaler_iterative import StandardScalerIterative

chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
scalerParams={"chunk_sizes":chunk_sizes}
scaler = StandardScalerIterative(**scalerParams)

def process_dataset(dataset, training: bool) -> DataLoader:
    coord_x=dataset.data['x-position']
    coord_y=dataset.data['y-position']
    surf_bool=dataset.extra_data['surface']
    position = np.stack([coord_x,coord_y],axis=1)

    nodes_features,node_labels=dataset.extract_data()
    if training:
        print("Normalize train data")
        nodes_features, node_labels = scaler.fit_transform(nodes_features, node_labels)
        print("Transform done")
    else:
        print("Normalize not train data")
        nodes_features, node_labels = scaler.transform(nodes_features, node_labels)
        print("Transform done")

    torchDataset=[]
    nb_nodes_in_simulations = dataset.get_simulations_sizes()
    start_index = 0
    # check alive
    t = dt.datetime.now()

    for nb_nodes_in_simulation in nb_nodes_in_simulations:
        #still alive?
        if dt.datetime.now() - t > dt.timedelta(seconds=60):
            print("Still alive - index : ", end_index)
            t = dt.datetime.now()
        end_index = start_index+nb_nodes_in_simulation
        simulation_positions = torch.tensor(position[start_index:end_index,:], dtype = torch.float) 
        simulation_features = torch.tensor(nodes_features[start_index:end_index,:], dtype = torch.float) 
        simulation_labels = torch.tensor(node_labels[start_index:end_index,:], dtype = torch.float) 
        simulation_surface = torch.tensor(surf_bool[start_index:end_index])

        sampleData=Data(pos=simulation_positions,
                        x=simulation_features, 
                        y=simulation_labels,
                        surf = simulation_surface.bool()) 
        torchDataset.append(sampleData)
        start_index += nb_nodes_in_simulation
    
    return DataLoader(dataset=torchDataset,batch_size=1)

In [33]:
processed_data = process_dataset(benchmark.train_dataset, training=True)

Normalize train data
Transform done


In [46]:
processed_data.dataset[0]

Data(x=[170180, 7], y=[170180, 4], pos=[170180, 2], surf=[170180])

In [40]:
processed_data.dataset[0]['x'].shape

torch.Size([170180, 7])

In [47]:
import airfrans

In [48]:
dataset_list, dataset_name = airfrans.dataset.load(root=DIRECTORY_NAME, task="scarce", train=True)

Loading dataset (task: scarce, split: train): 100%|██████████| 200/200 [02:18<00:00,  1.44it/s]


In [70]:
def extractMeanVar(dataset):
    with torch.no_grad():
        X, Y = torch.zeros(7).cuda(), torch.zeros(4).cuda()
        X, Y = X.double(), Y.double()
        tot = 0

        for xy in dataset:
            x, y = torch.Tensor(xy[:, 0:7]).cuda(), torch.Tensor(xy[:, 7:11]).cuda()
            X = X + x.sum(0)
            Y = Y + y.sum(0)
            tot += x.shape[0]

        X, Y = (X / tot).unsqueeze(0), (Y / tot).unsqueeze(0)

        sX, sY = torch.zeros(7).cuda(), torch.zeros(4).cuda()
        sX, sY = sX.double(), sY.double()

        for xy in dataset:
            x, y = torch.Tensor(xy[:, 0:7]).cuda(), torch.Tensor(xy[:, 7:11]).cuda()
            sX = ((x - X) ** 2).sum(0)
            sY = ((y - Y) ** 2).sum(0)

        sX, sY = torch.sqrt(sX / tot), torch.sqrt(sY / tot)
        sX, sY = sX.unsqueeze(0), sY.unsqueeze(0)

        return X.float(), sX.float(), Y.float(), sY.float()

extractMeanVar(dataset_list)

(tensor([[4.9988e-01, 1.1567e-02, 6.0934e+01, 4.9870e+00, 2.3195e-01, 1.6835e-03,
          8.4605e-05]], device='cuda:0'),
 tensor([[0.0499, 0.0221, 0.3601, 0.4114, 0.0331, 0.0025, 0.0043]],
        device='cuda:0'),
 tensor([[ 4.1659e+01,  9.9751e+00, -4.1020e+02,  8.1742e-04]], device='cuda:0'),
 tensor([[1.2067e+00, 1.0199e+00, 4.6566e+01, 7.1930e-05]], device='cuda:0'))

In [72]:
z = torch.zeros((3, 2))
f = lambda x: torch.cat([x, x], dim=1)

In [76]:
f(f(f(z)))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [62]:
@torch.no_grad()
def extractMeanVar(dataset):
    X, Y = torch.zeros(7).cuda(), torch.zeros(4).cuda()
    X, Y = X.double(), Y.double()
    tot = 0

    for data in dataset:
        x, y = data.x, data.y
        X = X + x.sum(dim=0)
        Y = Y + y.sum(dim=0)
        tot += x.shape[0]

    X, Y = (X / tot).unsqueeze(0), (Y / tot).unsqueeze(0)

    sX, sY = torch.zeros(7).cuda(), torch.zeros(4).cuda()
    sX, sY = sX.double(), sY.double()

    for xy in dataset:
        x, y = torch.Tensor(xy[:, 0:7]).cuda(), torch.Tensor(xy[:, 7:11]).cuda()
        sX = ((x - X) ** 2).sum(0)
        sY = ((y - Y) ** 2).sum(0)

    sX, sY = torch.sqrt(sX / tot), torch.sqrt(sY / tot)
    sX, sY = sX.unsqueeze(0), sY.unsqueeze(0)

    return X.float(), sX.float(), Y.float(), sY.float()

In [64]:
extractMeanVar(processed_data.dataset)

TypeError: unhashable type: 'slice'

In [61]:
dataset_list

[array([[ 4.21555805e+00,  6.31121755e-01,  3.59096884e+01, ...,
          2.85990310e+00,  5.03044717e-10,  0.00000000e+00],
        [ 4.21557093e+00,  6.31108701e-01,  3.59096884e+01, ...,
          2.86004496e+00,  5.04038977e-10,  0.00000000e+00],
        [ 3.99052548e+00,  5.86954355e-01,  3.59096884e+01, ...,
          3.25235939e+00,  3.09837572e-10,  0.00000000e+00],
        ...,
        [-2.13092208e+00, -1.53288102e+00,  3.59096884e+01, ...,
          2.22086563e+01,  3.12786264e-09,  0.00000000e+00],
        [-2.14492369e+00, -1.52218294e+00,  3.59096884e+01, ...,
          2.19026947e+01,  3.12771764e-09,  0.00000000e+00],
        [-2.15894747e+00, -1.51117182e+00,  3.59096884e+01, ...,
          2.15941086e+01,  3.12757775e-09,  0.00000000e+00]]),
 array([[ 4.21687031e+00, -1.99915648e-01,  5.87172837e+01, ...,
          2.49190092e+00,  7.00889854e-03,  0.00000000e+00],
        [ 4.21688938e+00, -1.99935317e-01,  5.87172837e+01, ...,
          2.49188542e+00,  7.00904103e

In [60]:
processed_data.dataset

[Data(x=[170180, 7], y=[170180, 4], pos=[170180, 2], surf=[170180]),
 Data(x=[179653, 7], y=[179653, 4], pos=[179653, 2], surf=[179653]),
 Data(x=[184381, 7], y=[184381, 4], pos=[184381, 2], surf=[184381]),
 Data(x=[177944, 7], y=[177944, 4], pos=[177944, 2], surf=[177944]),
 Data(x=[178145, 7], y=[178145, 4], pos=[178145, 2], surf=[178145]),
 Data(x=[178100, 7], y=[178100, 4], pos=[178100, 2], surf=[178100]),
 Data(x=[174285, 7], y=[174285, 4], pos=[174285, 2], surf=[174285]),
 Data(x=[174181, 7], y=[174181, 4], pos=[174181, 2], surf=[174181]),
 Data(x=[178475, 7], y=[178475, 4], pos=[178475, 2], surf=[178475]),
 Data(x=[180950, 7], y=[180950, 4], pos=[180950, 2], surf=[180950]),
 Data(x=[178007, 7], y=[178007, 4], pos=[178007, 2], surf=[178007]),
 Data(x=[178919, 7], y=[178919, 4], pos=[178919, 2], surf=[178919]),
 Data(x=[179267, 7], y=[179267, 4], pos=[179267, 2], surf=[179267]),
 Data(x=[180891, 7], y=[180891, 4], pos=[180891, 2], surf=[180891]),
 Data(x=[185397, 7], y=[185397, 4]